In [50]:
import pandas as pd 
from rapidfuzz import process, fuzz
import re
import unicodedata

In [ ]:
authors_institution_df = pd.read_csv('data_A_Z.csv')

authors_institution_df['share_pct'] = authors_institution_df['share_pct'].fillna(100)
authors_institution_df['institution'].value_counts().head(10)

isaure

In [30]:
top10_institutions = (
    authors_institution_df['institution']
    .value_counts()
    .head(10)
    .index
)

top10_institutions_df = authors_institution_df[
    authors_institution_df['institution'].isin(top10_institutions)
]

In [42]:
top10_institutions_df

,author_name,short_id,institution,share_pct,location,repec_institution_id,author_url
12,"Antipa, Pamfili",pan377,Banque de France,80%,"Paris, France",edi:bdfgvfr,https://ideas.repec.org/f/pan377.html
16,"Antman, Francisca M.",pan425,Institute of Labor Economics (IZA),5%,"Bonn, Germany",edi:izaaade,https://ideas.repec.org/f/pan425.html
58,"Antonioli, Federico",pan678,Joint Research Centre European Commission,100,"Sevilla, Spain",edi:ipjrces,https://ideas.repec.org/f/pan678.html
59,"Albanese, Giuseppe",pal552,Banca d'Italia,100,"Roma, Italy",edi:bdigvit,https://ideas.repec.org/f/pal552.html
94,"Albertazzi, Ugo",pal705,European Central Bank,50%,"Frankfurt am Main, Germany",edi:emieude,https://ideas.repec.org/f/pal705.html
...,...,...,...,...,...,...,...
29263,"Zweimuller, Josef",pzw2,Institute of Labor Economics (IZA),1%,"Bonn, Germany",edi:izaaade,https://ideas.repec.org/e/pzw2.html
29266,"Zwick, Thomas",pzw6,Leibniz-Zentrum für Europäische Wirtschaftsfor...,42%,"Mannheim, Germany",edi:zemande,https://ideas.repec.org/e/pzw6.html
29271,"Zerpa, Mariana",pze64,Institute of Labor Economics (IZA),10%,"Bonn, Germany",edi:izaaade,https://ideas.repec.org/e/pze64.html
29286,"Zylberberg, Andre",pzy2,Paris School of Economics,50%,"Paris, France",edi:eeparfr,https://ideas.repec.org/e/pzy2.html


In [ ]:
A_B_df = pd.read_csv('RePEc_Full_A_B.csv')
E_Z_df = pd.read_csv('RePEc_Full_E_Z.csv')
D_df = pd.read_csv('RePEc-D-not-full.csv')
S_Y_df = pd.read_csv('RePEc-from-S-to-Y-full.csv')
J_df = pd.read_csv('RePEc_J.csv')

In [40]:
JEL_df = pd.concat(
    [A_B_df, D_df, E_Z_df, S_Y_df, J_df],
    axis=0,
    ignore_index=True
)

JEL_df = JEL_df.drop_duplicates()

In [80]:
JEL_df

,JEL Subject,Title,Author(s),Journal,Year,Type,Affiliations,URL
0,A,Preparing students for careers using business ...,Nielsen Erland Hejn,NaN,2020.0,Working Paper,Erland Hejn Nielsen: Department of Economics a...,https://econpapers.repec.org/paper/aahaarhec/2...
1,A,Preparing students for careers using business ...,Nielsen Steen,NaN,2020.0,Working Paper,Erland Hejn Nielsen: Department of Economics a...,https://econpapers.repec.org/paper/aahaarhec/2...
2,A,"Measuring Democracy - Eight indices: Polity, F...",Paldam Martin,NaN,2021.0,Working Paper,Martin Paldam: Department of Economics and Bus...,https://econpapers.repec.org/paper/aahaarhec/2...
3,A,Oeconstudiet og den ÃÂ¸konomiske faggruppe ve...,Hylleberg Svend,NaN,2023.0,Working Paper,Svend Hylleberg: Department of Economics and B...,https://econpapers.repec.org/paper/aahaarhec/2...
4,A,Digital Tools in the Educational Environment E...,Andra Diaconescu,Research & Education,2024.0,Journal Article,Diaconescu Andra: Politehnica University of Ti...,https://econpapers.repec.org/article/aaijournl...
...,...,...,...,...,...,...,...,...
355589,K,A General Framework for Portfolio Theory. Part...,Zhu Qiji Jim,Risks,2018.0,Journal Article,Stanislaus Maier-Paape: Institut fÃ¼r Mathemat...,https://econpapers.repec.org/article/gamjrisks...
355590,K,Calendar Spread Exchange Options Pricing with ...,Hainaut Donatien,Risks,2018.0,Journal Article,"Donatien Hainaut: Institute of Statistics, Bio...",https://econpapers.repec.org/article/gamjrisks...
355591,K,On Fund Mapping Regressions Applied to Segrega...,Trottier Denis-Alexandre,Risks,2018.0,Journal Article,Denis-Alexandre Trottier: FacultÃ© des Science...,https://econpapers.repec.org/article/gamjrisks...
355592,K,On Fund Mapping Regressions Applied to Segrega...,Godin FrÃ©dÃ©ric,Risks,2018.0,Journal Article,Denis-Alexandre Trottier: FacultÃ© des Science...,https://econpapers.repec.org/article/gamjrisks...


In [ ]:
sum(JEL_df["Author(s)"].isna())
JEL_df = JEL_df.dropna(subset=["Author(s)"])

JEL_df = (
    JEL_df
        .assign(**{
            "Author(s)": JEL_df["Author(s)"].str.split(";")
        })
        .explode("Author(s)")
        .assign(**{
            "Author(s)": lambda x: x["Author(s)"].str.strip()
        })
        .reset_index(drop=True)
)

0

In [52]:
def strip_accents(s: str) -> str:
    s = unicodedata.normalize("NFKD", s)
    return "".join(ch for ch in s if not unicodedata.combining(ch))

def normalize_whitespace(s: str) -> str:
    return re.sub(r"\s+", " ", s).strip()

def standardize_name(raw):
    """
    Returns a cleaned full-name string with consistent formatting.
    Handles:
      - casefold/lowercase
      - accents removed
      - punctuation removed
      - 'Last, First' -> 'First Last'
      - extra spaces collapsed
    """
    if pd.isna(raw):
        return ""

    s = str(raw).strip()
    if s == "":
        return ""

    s = strip_accents(s).lower()

    # Convert "Last, First Middle" -> "First Middle Last"
    if "," in s:
        parts = [p.strip() for p in s.split(",", 1)]
        if len(parts) == 2 and parts[0] and parts[1]:
            s = f"{parts[1]} {parts[0]}"

    # Replace punctuation with spaces (keeps letters/numbers)
    s = re.sub(r"[^\w\s]", " ", s)

    # Collapse whitespace
    s = normalize_whitespace(s)

    return s

def author_key(raw):
    """
    Robust matching key: "<lastname> <firstinitials>"
    Example: "John Ronald Reuel Tolkien" -> "tolkien jrr"
    """
    s = standardize_name(raw)
    if not s:
        return ""

    parts = s.split()
    if len(parts) == 1:
        return parts[0]

    last = parts[-1]
    initials = "".join(p[0] for p in parts[:-1] if p)
    return f"{last} {initials}"


In [ ]:
JEL_df_std = pd.DataFrame({
    "author_std": JEL_df["Author(s)"].apply(standardize_name)
})

JEL_df_std = JEL_df.copy()

JEL_df_std["author_std"] = (
    JEL_df_std["Author(s)"]
        .astype(str)
        .apply(standardize_name)
)

JEL_df_std = JEL_df_std.drop(columns="Author(s)")

In [79]:
JEL_df_std

,JEL Subject,Title,Journal,Year,Type,Affiliations,URL,author_std,matched_author,match_score,matched_index
95460,D,"Informal Markets, Domestic Production and Dema...",Economics Bulletin,2017.0,Journal Article,Armagan Aktuna-gunes: Paris School of Economic...,https://econpapers.repec.org/article/eblecbull...,gardes francois,"Gardes, francois",90.322581,1154.0
316378,J,"Informal Markets, Domestic Production and Dema...",Economics Bulletin,2017.0,Journal Article,Armagan Aktuna-gunes: Paris School of Economic...,https://econpapers.repec.org/article/eblecbull...,gardes francois,"Gardes, francois",90.322581,1154.0
288704,Z,Testing for Peer Effects Using Genetic Data,NaN,2018.0,Working Paper,NaN,https://econpapers.repec.org/paper/yorhectdg/1...,han,"Abeler, Johannes",90.000000,22.0
189142,E,"Micro, Small, and Medium Enterprisesâ Readin...",Economies,2023.0,Journal Article,"Lina Anatan: Faculty of Business, Maranatha Ch...",https://econpapers.repec.org/article/gamjecomi...,nur,"Altindag, Onur",90.000000,58.0
307126,J,CRISIS OF MEDICAL STAFF EMPLOYMENT WITH HIGHER...,Annales Universitatis Apulensis Series Oeconomica,2018.0,Journal Article,NaN,https://econpapers.repec.org/article/alujournl...,e,"Chen, Xi",90.000000,545.0
...,...,...,...,...,...,...,...,...,...,...,...
324384,J,MarchÃ© du travail: institutions et rÃ©formes ...,Bulletin de la Banque de France,2014.0,Journal Article,NaN,https://econpapers.repec.org/article/bfrbullbf...,,NaN,0.000000,NaN
324383,J,Le comportement des entreprises durant la cris...,Bulletin de la Banque de France,2014.0,Journal Article,NaN,https://econpapers.repec.org/article/bfrbullbf...,,NaN,0.000000,NaN
84574,D,Firm and Worker Dynamics in a Frictional Labor...,NaN,2019.0,Working Paper,NaN,https://econpapers.repec.org/paper/cprceprdp/1...,,NaN,0.000000,NaN
203382,E,Austerity and Public debt Dynamics,NaN,2019.0,Working Paper,NaN,https://econpapers.repec.org/paper/cprceprdp/1...,,NaN,0.000000,NaN


In [60]:
#Do fuzzy matching for the authors in df_Authors to find the match in df_A_B_Code
#Obtain matching scores 
choices = (
    top10_institutions_df["author_name"]
        .astype(str)
        .unique()
        .tolist()
)

def fuzzy_match_one(name):
    if pd.isna(name) or str(name).strip() == "":
        return pd.Series(
            [None, 0, None],
            index=["matched_author", "match_score", "matched_index"]
        )

    match, score, idx = process.extractOne(
        str(name),
        choices,
        scorer=fuzz.WRatio
    )

    return pd.Series(
        [match, score, idx],
        index=["matched_author", "match_score", "matched_index"]
    )

JEL_df_std[["matched_author", "match_score", "matched_index"]] = (
    JEL_df_std["author_std"].apply(fuzzy_match_one)
)

In [81]:
JEL_df_std = JEL_df_std.sort_values(
    by="match_score",
    ascending=False
)

JEL_df_std
JEL_df_std[JEL_df_std["match_score"] >= 77]


,JEL Subject,Title,Journal,Year,Type,Affiliations,URL,author_std,matched_author,match_score,matched_index
95460,D,"Informal Markets, Domestic Production and Dema...",Economics Bulletin,2017.0,Journal Article,Armagan Aktuna-gunes: Paris School of Economic...,https://econpapers.repec.org/article/eblecbull...,gardes francois,"Gardes, francois",90.322581,1154.0
316378,J,"Informal Markets, Domestic Production and Dema...",Economics Bulletin,2017.0,Journal Article,Armagan Aktuna-gunes: Paris School of Economic...,https://econpapers.repec.org/article/eblecbull...,gardes francois,"Gardes, francois",90.322581,1154.0
77724,B,The Education-Health Nexus: A Meta-Analysis,NaN,2020.0,Working Paper,NaN,https://econpapers.repec.org/paper/yorhectdg/2...,x,"Lay, Max",90.000000,1705.0
48673,B,Glimpses of Henry Schultz in Mussolini?s 1934 ...,HISTORY OF ECONOMIC THOUGHT AND POLICY,2015.0,Journal Article,NaN,https://econpapers.repec.org/article/fanspespe...,n d,"Van der Vorst, Camille",90.000000,2934.0
312773,J,Reparto de la riqueza. Reparto del empleo,EKONOMIAZ. Revista vasca de EconomÃ­a,1996.0,Journal Article,NaN,https://econpapers.repec.org/article/ekzekonoz...,lab,"Colabella, Andrea",90.000000,482.0
...,...,...,...,...,...,...,...,...,...,...,...
191915,E,Exchange rate forecasts and expected fundamentals,NaN,2014.0,Working Paper,NaN,https://econpapers.repec.org/paper/zbwifwkwp/1...,dick christian,"Dick, Christian David",77.142857,792.0
289546,Z,The Iron Curtain and Referee Bias in Internati...,Journal of Sports Economics,2024.0,Journal Article,NaN,https://econpapers.repec.org/article/saejospec...,singleton carl,"Singleton, Carl Andrew",77.142857,2717.0
43501,A,The renewable energy consumption and growth in...,NaN,2018.0,Working Paper,NaN,https://econpapers.repec.org/paper/pramprapa/8...,ozdemir zeynel,"Ozdemir, Zeynel Abidin",77.142857,2115.0
1490,A,Analysis of Factors Influencing the Motivation...,European Research Studies Journal,2021.0,Journal Article,NaN,https://econpapers.repec.org/article/ersjournl...,sadlowska wrzesinska joanna,"Osinska, Joanna",77.142857,2082.0
